In [1]:
from preprocess_data import *
from config import config_base
import time
import utils
from data_pre import title_question
config = config_base.config

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.442 seconds.
Prefix dict has been built succesfully.


In [2]:
df = organize_data(config.train_data)
df = df.sample(2000, random_state=1)

In [3]:
df = deal_data(df)

In [4]:
df = shorten_content_all(df, config.max_len)

答案存在比例：0.9775
截取比例：0.9725
截取准确率：0.9949
max length: 512
min length: 29
mean length:282
median length:274


In [5]:
df = build_answer_range(df)

answer generation accuracy(all): 0.9846
wrong split: 0.0154


In [ ]:
df = select_data(df)

In [14]:
df[df['for_train']]

,article_id,article_type,article_title,article_content,question_id,article_question,article_answer,question_type,title,content,question,answer,shorten_content,is_in,len,answer_start,answer_end,for_train
9126,45549,防务快讯,美媒:东风-21D导弹能让美航母变成扭曲的金属块,环球网综合报道：美国《防务新闻》周刊网站7月23日报道称，美国国会研究所（U.S. Con...,986e90ea-c170-4268-abf1-c79026543a9f,从上世纪90年中期到现在，中国海军购买了多少艘俄制基洛级攻击潜艇,12,数值型问题,美媒:东风-21D导弹能让美航母变成扭曲的金属块,环球网综合报道：美国《防务新闻》周刊网站7月23日报道称，美国国会研究所（U.S. Cong...,从上世纪90年中期到现在，中国海军购买了多少艘俄制基洛级攻击潜艇,12,环球网综合报道：美国《防务新闻》周刊网站7月23日报道称，美国国会研究所（U.S. Cong...,True,407,186.0,186.0,True
24014,26328,新闻,台媒猜测解放军绕台常态化：对台进行军事施压,【环球时报综合报道】岛内媒体近日纷纷猜测，未来解放军可能将加强绕台实战化巡航，对台进行军事施...,be61eb23-43bf-4c17-9732-590db3f4c119,谁是刘锐,曾驾驶轰-6K赴西太平洋训练的飞行员,事实型问题,台媒猜测解放军绕台常态化：对台进行军事施压,【环球时报综合报道】岛内媒体近日纷纷猜测，未来解放军可能将加强绕台实战化巡航，对台进行军事施...,谁是刘锐,曾驾驶轰-6K赴西太平洋训练的飞行员,【环球时报综合报道】岛内媒体近日纷纷猜测，未来解放军可能将加强绕台实战化巡航，对台进行军事施...,True,375,82.0,91.0,True
83950,45999,防务快讯,美国海军决定推迟舰载无人作战飞机招标书发布和飞机部署时间,[据美国海军协会网站2015年2月2日报道]2015年2月2日下午，在2016财年（译者注：...,49d3bf88-2811-4e8a-9c96-a2e30761d221,美国国防部海军部主管预算工作的副助理部长是谁,威廉雷斯基尔少将,事实型问题,美国海军决定推迟舰载无人作战飞机招标书发布和飞机部署时间,[据美国海军协会网站2015年2月2日报道]2015年2月2日下午，在2016财年（译者注：...,美国国防部海军部主管预算工作的副助理部长是谁,威廉雷斯基尔少将,[据美国海军协会网站2015年2月2日报道]2015年2月2日下午，在2016财年（译者注：...,True,354,72.0,76.0,True
28738,23081,新闻,习近平会见法国总统马克龙,人民日报北京1月8日电（本报记者李伟红）国家主席习近平8日在钓鱼台国宾馆会见来华进行国事访问...,cc547336-e8eb-4f4a-813e-8e438ad0f6bc,国家主席习近平8日在哪里会见来华进行国事访问的法国总统马克龙,钓鱼台国宾馆,事实型问题,习近平会见法国总统马克龙,人民日报北京1月8日电（本报记者李伟红）国家主席习近平8日在钓鱼台国宾馆会见来华进行国事访问...,国家主席习近平8日在哪里会见来华进行国事访问的法国总统马克龙,钓鱼台国宾馆,人民日报北京1月8日电（本报记者李伟红）国家主席习近平8日在钓鱼台国宾馆会见来华进行国事访问...,True,393,26.0,26.0,True
69638,6278,新闻,不走了！美军将长期驻军叙利亚提防“伊斯兰国2.0”,图为在叙利亚的美军（来源：海外网） 美国国防部长詹姆斯马蒂斯13日在五角大楼说，美军将长...,c1a2c146-d5d6-4a44-8ffd-e821969c92f5,叙利亚政府军及其盟军、美国支持的叙反对派武装仍在与谁交战？,“伊斯兰国”,事实型问题,不走了！美军将长期驻军叙利亚提防“伊斯兰国2.0”,图为在叙利亚的美军（来源：海外网） 美国国防部长詹姆斯马蒂斯13日在五角大楼说，美军将长期在...,叙利亚政府军及其盟军、美国支持的叙反对派武装仍在与谁交战？,“伊斯兰国”,图为在叙利亚的美军（来源：海外网） 美国国防部长詹姆斯马蒂斯13日在五角大楼说，美军将长期在...,True,282,206.0,209.0,True
86822,35364,防务快讯,美反导系统紧盯俄核导弹 双方上演对决 (2006/01/06),最近几天，美国的反导系统与俄罗斯战略导弹展开了暗中较量。据英国《苏格兰人报》报道，普京日前下...,2a52ca1f-9980-4913-8819-84a054158ddf,12月21日，美国防部宣布，美国已经部署了美国反导系统中的第几枚陆基中段拦截导弹,10,数值型问题,美反导系统紧盯俄核导弹 双方上演对决 (2006/01/06),最近几天，美国的反导系统与俄罗斯战略导弹展开了暗中较量。据英国《苏格兰人报》报道，普京日前下...,12月21日，美国防部宣布，美国已经部署了美国反导系统中的第几枚陆基中段拦截导弹,10,最近几天，美国的反导系统与俄罗斯战略导弹展开了暗中较量。据英国《苏格兰人报》报道，普京日前下...,True,202,85.0,85.0,True
3023,44094,防务快讯,"短评：台军F-16""采购变升级""后面的瞒天过海",新华网专稿（新华军事评论员 郑文浩）美国国会一名官员16日称，美国已经决定不向台湾出售新一代...,3ffa07f3-c624-4e22-9f9a-ce9d186b490e,美国国会一名官员16日称什么,美国已经决定不向台湾出售新一代的F-16战机，而会帮助台湾升级后者已经拥有的F-16战机,观点型问题,"短评：台军F-16""采购变升级""后面的瞒天过海",新华网专稿（新华军事评论员 郑文浩）美国国会一名官员16日称，美国已经决定不向台湾出售新一代...,美国国会一名官员16日称什么,美国已经决定不向台湾出售新一代的F-16战机，而会帮助台湾升级后者已经拥有的F-16战机,新华网专稿（新华军事评论员 郑文浩）美国国会一名官员16日称，美国已经决定不向台湾出售新一代...,True,196,34.0,60.0,True
7146,7215,新闻,协助反恐！ 俄罗斯为菲律宾提供5000支枪,（路透社图） 【环球网报道 记者 田瑞哲】据新加坡《联合早报》报道，10月12日，俄罗斯驻...,3c998d1f-b0a0-4fdc-a148-d3d0e2fb900b,俄罗斯提供的军备将于本月22日抵达何处,马尼拉,事实型问题,协助反恐！ 俄罗斯为菲律宾提供5000支枪,（路透社图） 【环球网报道 记者 田瑞哲】据新加坡《联合早报》报道，10月12日，俄罗斯驻马...,俄罗斯提供的军备将于本月22日抵达何处,马尼拉,（路透社图） 【环球网报道 记者 田瑞哲】据新加坡《联合早报》报道，10月12日，俄罗斯驻马...,True,193,183.0,183.0,True
20855,49903,防务快讯,东部战区陆军：用上国产安全云,东部战区陆军机关成规模部署运用基于麒麟操作系统的办公系统，国产自主，安全可靠，摆脱了对国外系...,c6d41795-0bc7-4edb-a80c-fad621aaf012,党委首长就明确提出要坚决贯彻落实党中央、中央军委和习主席指示要求是什么,从我做起、超前布局，勇于实践、大胆探索，坚定不移推进自主可控信息系统建设。,篇章型问题,东部战区陆军：用上国产安全云,东部战区陆军机关成规模部署运用基于麒麟操作系统的办公系统，国产自主，安全可靠，摆脱了对国外系...,党委首长就明确提出要坚决贯彻落实党中央、中央军委和习主席指示要求是什么,从我做起、超前布局，勇于实践、大胆探索，坚定不移推进自主可控信息系统建设,东部战区陆军机关成规模部署运用基于麒麟操作系统的办公系统，国产自主，安全可靠，摆脱了对国外系...,True,475,257.0,273.0,True
58413,51890,防务快讯,提供最大的灵活性，获得最佳的解决方案――美陆军炮兵指挥系统的成功经验,先进野战炮兵战术数据系统（AFATDS）为美陆军和海军陆战队提供重要的火力指挥和控制能力。自...,e9921d5d-b756-4419-8fa6-689659e9541a,陆军采用了商业IT和软件中的最佳方法，并提出哪些创新的方法从供应商处获得潜在的解决方案,首先，陆军需要一种能力，但不会指导如何实现该能力，因此仅仅发布一个概述了现代化总体目标的声明...,列表型问题,提供最大的灵活性，获得最佳的解决方案――美陆军炮兵指挥系统的成功经验,先进野战炮兵战术数据系统（AFATDS）为美陆军和海军陆战队提供重要的火力指挥和控制能力。自...,陆军采用了商业IT和软件中的最佳方法，并提出哪些创新的方法从供应商处获得潜在的解决方案,首先，陆军需要一种能力，但不会指导如何实现该能力，因此仅仅发布一个概述了现代化总体目标的声明...,先进野战炮兵战术数据系统（AFATDS）为美陆军和海军陆战队提供重要的火力指挥和控制能力。自...,False,309,NaN,NaN,True
